In [1]:
# naive prediction model
# uses only elo in a hard coded logic model

import archives_manager

In [26]:
def archive_filter(archived_game):
    if archived_game['rated'] == False:
        return False
    
    if archived_game['time_class'] != 'rapid':
        return False

    white_result = archived_game['white']['result']
    black_result = archived_game['black']['result']
    if white_result == "win" or black_result == "win":
        return True

    return False

In [27]:
player_name = "BIG_TONKA_T"

# get 1000 recent rated win/loss games
recent_archive = archives_manager.getMostRecentGames(player_name, 1000, filter_func=archive_filter)

recent_archive[0]

{'url': 'https://www.chess.com/game/live/93900677605',
 'pgn': '[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2023.11.16"]\n[Round "-"]\n[White "UnderTheBeer"]\n[Black "BIG_TONKA_T"]\n[Result "0-1"]\n[CurrentPosition "r1bq1rk1/1pp2ppp/5n2/p2PN3/3bP3/P2B4/5PPP/RN1Q1RK1 w - -"]\n[Timezone "UTC"]\n[ECO "C00"]\n[ECOUrl "https://www.chess.com/openings/French-Defense-Steiner-Variation"]\n[UTCDate "2023.11.16"]\n[UTCTime "19:11:26"]\n[WhiteElo "1434"]\n[BlackElo "1501"]\n[TimeControl "600"]\n[Termination "BIG_TONKA_T won by resignation"]\n[StartTime "19:11:26"]\n[EndDate "2023.11.16"]\n[EndTime "19:14:46"]\n[Link "https://www.chess.com/game/live/93900677605"]\n\n1. e4 {[%clk 0:10:00]} 1... e6 {[%clk 0:10:00]} 2. c4 {[%clk 0:09:56.1]} 2... d6 {[%clk 0:09:59.4]} 3. d4 {[%clk 0:09:53.5]} 3... Ne7 {[%clk 0:09:57.4]} 4. Nf3 {[%clk 0:09:52.1]} 4... Ng6 {[%clk 0:09:56.4]} 5. Bd3 {[%clk 0:09:49.2]} 5... e5 {[%clk 0:09:49.6]} 6. d5 {[%clk 0:09:40.6]} 6... Nd7 {[%clk 0:09:48.2]} 7. b4 {[%clk 0:09:33

In [29]:
recent_archive[-1]

{'url': 'https://www.chess.com/game/live/90924579037',
 'pgn': '[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2023.10.13"]\n[Round "-"]\n[White "Kythadrln"]\n[Black "BIG_TONKA_T"]\n[Result "1-0"]\n[CurrentPosition "r1bqr1k1/ppp1bNpp/3p4/3Q3n/4P3/P4P2/P1P3PP/R1B2RK1 w - -"]\n[Timezone "UTC"]\n[ECO "C00"]\n[ECOUrl "https://www.chess.com/openings/French-Defense-Normal-Variation"]\n[UTCDate "2023.10.13"]\n[UTCTime "08:27:57"]\n[WhiteElo "1360"]\n[BlackElo "1360"]\n[TimeControl "600"]\n[Termination "Kythadrln won by resignation"]\n[StartTime "08:27:57"]\n[EndDate "2023.10.13"]\n[EndTime "08:31:19"]\n[Link "https://www.chess.com/game/live/90924579037"]\n\n1. d4 {[%clk 0:10:00]} 1... e6 {[%clk 0:10:00]} 2. e4 {[%clk 0:09:58.6]} 2... d6 {[%clk 0:09:59.5]} 3. Bc4 {[%clk 0:09:48.6]} 3... Nd7 {[%clk 0:09:58.7]} 4. d5 {[%clk 0:09:43.8]} 4... Nb6 {[%clk 0:09:56]} 5. dxe6 {[%clk 0:09:34.5]} 5... Nxc4 {[%clk 0:09:53.6]} 6. exf7+ {[%clk 0:09:31.6]} 6... Kxf7 {[%clk 0:09:50.4]} 7. Na3 {[%clk 0:09:19

In [37]:
def get_elo(archived_game):
    white_player = archived_game['white']['username']
    black_player = archived_game['black']['username']

    white_elo = archived_game['white']['rating']
    black_elo = archived_game['black']['rating']

    player_elo = 0
    opponent_elo = 0

    if white_player == player_name:
        player_elo = white_elo
        opponent_elo = black_elo
    elif black_player == player_name:
        player_elo = black_elo
        opponent_elo = white_elo
    else:
        raise ValueError(f"Player name '{player_name}' does not match either player in the game.")
    
    return {
        'Player': player_elo,
        'Opponent': opponent_elo
    }

def get_average_elo(archive):
    n = len(archive)
    avg = 0

    for archived_game in archive:
        avg += (get_elo(archived_game)['Player'] / n)

    return avg

def predict_win(archived_game):
    elo = get_elo(archived_game)

    if elo['Player'] > elo['Opponent']:
        return 1
    else:
        return 0

def actual_win(archived_game):
    white_player = archived_game['white']['username']
    black_player = archived_game['black']['username']

    white_result = archived_game['white']['result']
    black_result = archived_game['black']['result']

    if white_player == player_name:
        if white_result == 'win':
            return 1
        elif black_result == 'win':
            return 0
        else:
            raise ValueError(f"Archived game did not result in a win for either player")
    elif black_player == player_name:
        if white_result == 'win':
            return 0
        elif black_result == 'win':
            return 1
        else:
            raise ValueError(f"Archived game did not result in a win for either player")

    raise ValueError(f"Player name '{player_name}' does not match either player in the game.")

In [31]:
import pandas as pd

def create_dataframe(recent_archive):
    lst = []

    for archived_game in recent_archive:
        pred = predict_win(archived_game)
        actual = actual_win(archived_game)

        elo = get_elo(archived_game)

        lst.append({
            'Player_Elo': elo['Player'],
            'Opponent_Elo': elo['Opponent'],
            'Predicted': pred,
            'Actual': actual
        })

    df = pd.DataFrame(lst)

    return df

In [32]:
df = create_dataframe(recent_archive)
df.head(10)

,Player_Elo,Opponent_Elo,Predicted,Actual
0,1501,1434,1,1
1,1494,1437,1,1
2,1487,1493,0,1
3,1478,1555,0,1
4,1468,1459,1,1
5,1460,1523,0,1
6,1450,1380,1,0
7,1460,1524,0,0
8,1467,1458,1,0
9,1476,1493,0,0


In [33]:
def df_stats(df):
    TP = len(df[(df['Predicted'] == 1) & (df['Actual'] == 1)])
    TN = len(df[(df['Predicted'] == 0) & (df['Actual'] == 0)])
    FP = len(df[(df['Predicted'] == 1) & (df['Actual'] == 0)])
    FN = len(df[(df['Predicted'] == 0) & (df['Actual'] == 1)])

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    return {
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN,
        'accuracy': accuracy
    }

In [34]:
print(df_stats(df))

{'TP': 369, 'TN': 321, 'FP': 168, 'FN': 142, 'accuracy': 0.69}


In [46]:
import numpy as np

players = [
    'BIG_TONKA_T',
    'UnderTheBeer',
    'Dandres0_0',
    'Ale9800',
    'MrOGH'
]

stats_list = []

for player in players:
    player_name = player

    recent_archive = archives_manager.getMostRecentGames(player_name, 1000, filter_func=archive_filter)
    df = create_dataframe(recent_archive)
    
    player_info = {
        'Name': player_name,
        'Current Rating': get_elo(recent_archive[0])['Player'],
        'Average Rating': round(get_average_elo(recent_archive))
    }

    player_info.update(df_stats(df))

    stats_list.append(player_info)

stats_df = pd.DataFrame(stats_list)

print('overall average accuracy', round(np.mean(stats_df['accuracy']), 3))

stats_df

overall average accuracy 0.665


,Name,Current Rating,Average Rating,TP,TN,FP,FN,accuracy
0,BIG_TONKA_T,1501,1384,369,321,168,142,0.690
1,UnderTheBeer,1424,1343,349,308,178,165,0.657
2,Dandres0_0,1455,1499,352,320,173,155,0.672
3,Ale9800,1467,1488,336,303,188,173,0.639
4,MrOGH,1362,1325,344,321,167,168,0.665
